In [ ]:
# New try without obs and local minima and maxima yet

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import warnings
import numpy as np

warnings.filterwarnings("ignore")

class WL(Strategy):
    atr_window = 13
    tradehourStart = 0
    order = 10
    longs = False
    shorts = True
    tradehourEnd = 23
    candlecountEntry = 10
    def init(self):
        self.vwap = self.I(VWAP2, self.data.df, 0)
        self.vwapu1 = self.I(VWAP2, self.data.df, 1)
        self.ema = self.I(ta.ema, self.data.df["Close"], length=100)
        self.vwapl1 = self.I(VWAP2, self.data.df, 2)
        self.vwapu2 = self.I(VWAP2, self.data.df, 3)
        self.lastdiffs = []
        self.vwapl2 = self.I(VWAP2, self.data.df, 4)
        self.vwapu3 = self.I(VWAP2, self.data.df, 5)
        self.vwapl3 = self.I(VWAP2, self.data.df, 6)
        self.atr = self.I(atr, self.data.df, self.atr_window)
        self.trade_opened = False
        self.sholong = 0
        self.pivotLower = self.I(pivotPoints, self.data.df, self.order, 0)
        self.pivotUpper = self.I(pivotPoints, self.data.df, self.order, 1)
        self.tagged = False
        self.taggeddev = 0
        self.idk = self.data.df
        self.wavetrendMirror = self.I(wavetrend3d, self.idk, 5,False,"30T", True)
        self.wavetrend = self.I(wavetrend3d, self.idk, 5,False ,"30T", False)
        self.stoploss = 0
        self.candlecount = 0
        self.prevvalueU = 0
        self.prevvalueB = 0
        self.taggedcounter = 0
        self.hochpunkte = []
        self.tiefpunkte = []
        self.supertrendu = self.I(supertrend, self.data.df, 10, 2.5, 0)
        self.supertrendl = self.I(supertrend, self.data.df, 10, 2.5, 1)
        

    def next(self):
        if(self.pivotUpper[-1] > 0 and self.pivotUpper[-1] != self.prevvalueU):
            lower_val = 100000
            higher_val = 0
            for i in range(1, 3):
                if(self.data.Low[-i] < lower_val):
                    lower_val = self.data.Low[-i]
                if(self.data.High[-i] > higher_val):
                    higher_val = self.data.High[-i]
            self.hochpunkte.append([higher_val, higher_val, self.candlecount])
            self.prevvalueU = self.pivotUpper[-1]
        if(self.pivotLower[-1] > 0 and self.pivotLower[-1] != self.prevvalueB):
            lower_val = 100000
            higher_val = 0
            for i in range(1, 3):
                if(self.data.Low[-i] < lower_val):
                    lower_val = self.data.Low[-i]
                if(self.data.High[-i] > higher_val):
                    higher_val = self.data.High[-i]
            self.tiefpunkte.append([higher_val, lower_val, self.candlecount])
            self.prevvalueB = self.pivotLower[-1]

        
        if not self.trade_opened and self.data.index[-1].hour >= self.tradehourStart and self.data.index[-1].hour <= self.tradehourEnd:
            if((self.data.High[-1] > self.vwapu2[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)) or (self.data.High[-1] > self.vwapu3[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu3[-1] + (self.atr[-1]/2)) and self.shorts):
                if self.wavetrendMirror[-1] > self.wavetrend[-1]:
                    self.tagged = True
                    if(self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)):
                        self.taggeddev = 2
                    else:
                        self.taggeddev = 3
                    self.sholong = 1
            elif((self.data.Low[-1] < self.vwapl2[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)) or (self.data.Low[-1] < self.vwapl3[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl3[-1] - (self.atr[-1]/2)) and self.longs):
                if self.wavetrendMirror[-1] < self.wavetrend[-1]:
                    self.tagged = True
                    if(self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)):
                        self.taggeddev = 2
                    else:
                        self.taggeddev = 3
                    self.sholong = 0
            elif (self.tagged == True and self.sholong == 1):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.High[-1] > self.vwapu2[-1] + (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.High[-1] > self.vwapu3[-1] + (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                if self.taggedcounter < self.candlecountEntry:
                    if ha_open > (ha_high - ((ha_high - ha_low)*0.12) and self.data.Close[-1]>self.vwapu1[-1]):
                        self.trade_opened = True
                        self.stoploss = self.data.Close[-1] + (self.atr[-1]/2)
                        def berechnen(kapital, stoploss, entry):
                            diff = abs(stoploss - entry)
                            if(diff < 0.000001):
                                diff = 0
                            self.lastdiffs.append(diff)
                            if(diff == 0):
                                size = 0
                            else: 
                                size = abs((kapital/10000) / abs(diff))
                            return int(round(size,0))
                        sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                        self.sell(size=sizeO)
                    self.taggedcounter += 1
                else: 
                    self.tagged = False
                    self.taggedcounter = 0                    
            elif (self.tagged == True and self.sholong == 0):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.Low[-1] < self.vwapl2[-1] - (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.Low[-1] < self.vwapl3[-1] - (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                else:
                    if self.taggedcounter < self.candlecountEntry:
                        if ha_open < (ha_low + (ha_high - ha_low) * 0.12) and self.data.Close[-1]<self.vwapl1[-1]:
                            self.stoploss = self.data.Close[-1] - (self.atr[-1]/2)
                            self.trade_opened = True
                            def berechnen(kapital, stoploss, entry):
                                diff = abs(stoploss - entry)
                                if(diff < 0.000001):
                                    diff = 0
                                self.lastdiffs.append(diff)
                                if(diff == 0):
                                    size = 0
                                else: 
                                    size = abs((kapital/10000) / abs(diff))
                                return int(round(size,0))
                            sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                            self.buy(size=sizeO)
                        else: 
                            self.tagged = False
                            self.taggedcounter = 0
                        self.taggedcounter += 1
        elif self.trade_opened:
            if self.sholong == 0:
                if(self.supertrendl[-2] > 0 and self.supertrendu[-1] > 0) or self.data.Close[-1] < self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
            elif self.sholong == 1:
                if(self.supertrendu[-2] > 0 and self.supertrendl[-1] > 0) or self.data.Close[-1] > self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
